<a href="https://colab.research.google.com/github/fadhluibnu/ANOMALY_IOT_NETWORK_DETECTION/blob/main/2_IOT_ANOMALI_DETECTION_GROX_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>